# 🎯 감정 분석 모델 최적화: KoBERT 파인튜닝

이 노트북은 **KoBERT 기반 감정 분석 모델**을 최고 성능으로 학습시키기 위한 완전한 파이프라인을 제공합니다.

## 주요 특징
- ✅ **데이터 품질 개선**: 불균형 해소, 전처리 최적화
- ✅ **KoBERT 파인튜닝**: 한국어 사전학습 모델 활용
- ✅ **K-Fold 교차 검증**: 일반화 성능 향상
- ✅ **하이퍼파라미터 튜닝**: 최적 학습률, 배치 크기 등
- ✅ **성능 모니터링**: 실시간 메트릭 추적


## 📦 1. 필수 라이브러리 설치


In [ ]:
# 필수 라이브러리 설치
# 의존성 충돌 방지를 위해 호환되는 버전으로 설치
!pip install --upgrade pip -q
!pip install transformers>=4.40.0 -q
!pip install huggingface-hub>=0.20.0 -q
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install datasets accelerate -q
!pip install scikit-learn imbalanced-learn -q
!pip install pandas numpy matplotlib seaborn -q
!pip install tqdm -q

print("✅ 모든 라이브러리 설치 완료!")


## 📚 2. 라이브러리 임포트


In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from typing import Dict, List, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

# 머신러닝
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# 딥러닝
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from datasets import Dataset as HFDataset

# 진행상황 표시
from tqdm import tqdm
tqdm.pandas()

print("✅ 라이브러리 임포트 완료!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"💻 CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")

# transformers 버전 확인
try:
    import transformers
    print(f"📦 Transformers 버전: {transformers.__version__}")
except:
    print("⚠️ Transformers 버전 확인 실패")


## ⚙️ 3. 설정 (Configuration)


In [ ]:
# ========== 모델 설정 ==========
MODEL_NAME = "monologg/kobert"  # KoBERT 모델
# 다른 옵션: "monologg/koelectra-base-v3-discriminator" (KoELECTRA)

# ========== 데이터 설정 ==========
DATA_PATH = "/content/emotion_corpus_with_kote.csv"  # Colab에 업로드할 파일 경로
MAX_LENGTH = 128  # 최대 토큰 길이 (BERT는 보통 128 또는 512)

# ========== 학습 설정 ==========
LEARNING_RATE = 2e-5  # 파인튜닝용 낮은 학습률 (2e-5 ~ 5e-5 권장)
BATCH_SIZE = 16  # GPU 메모리에 맞게 조정 (16, 32, 64 등)
NUM_EPOCHS = 5  # 에포크 수 (과적합 방지를 위해 3-5 권장)
WEIGHT_DECAY = 0.01  # 정규화
WARMUP_STEPS = 500  # 워밍업 스텝

# ========== K-Fold 설정 ==========
N_FOLDS = 5  # K-Fold 교차 검증 폴드 수
RANDOM_SEED = 42  # 재현성을 위한 시드

# ========== 데이터 불균형 해소 설정 ==========
USE_SAMPLING = True  # 샘플링 사용 여부
SAMPLING_STRATEGY = "balanced"  # 'balanced', 'minority', 또는 비율 지정

# ========== 감정 클래스 매핑 ==========
LABEL2ID = {'joy': 0, 'sad': 1, 'anxiety': 2, 'anger': 3, 'neutral': 4}
ID2LABEL = {v: k for k, v in LABEL2ID.items()}
NUM_LABELS = len(LABEL2ID)

print("✅ 설정 완료!")
print(f"📊 모델: {MODEL_NAME}")
print(f"📏 최대 길이: {MAX_LENGTH}")
print(f"🎯 학습률: {LEARNING_RATE}")
print(f"📦 배치 크기: {BATCH_SIZE}")
print(f"🔄 에포크: {NUM_EPOCHS}")
print(f"🔀 K-Fold: {N_FOLDS}")


## 📤 4. 데이터 업로드

**Colab에서 실행 시**: 아래 셀을 실행하여 로컬 파일을 업로드하세요.


In [ ]:
from google.colab import files

# 파일 업로드 (Colab에서만 실행)
uploaded = files.upload()

# 업로드된 파일명 확인
for filename in uploaded.keys():
    print(f"✅ 업로드 완료: {filename}")
    # 파일명을 emotion_corpus_with_kote.csv로 변경 (선택사항)
    if filename != "emotion_corpus_with_kote.csv":
        os.rename(filename, "emotion_corpus_with_kote.csv")
        print(f"   → emotion_corpus_with_kote.csv로 이름 변경")

DATA_PATH = "/content/emotion_corpus_with_kote.csv"


## 🧹 5. 데이터 전처리 및 품질 개선


In [ ]:
def clean_text(text: str) -> str:
    """
    텍스트 정제 함수
    - HTML 태그 제거
    - URL 제거
    - 이메일 제거
    - 불필요한 공백 제거
    """
    if pd.isna(text) or text == "":
        return ""
    
    text = str(text)
    
    # HTML 태그 제거
    text = re.sub(r'<[^>]+>', '', text)
    
    # URL 제거
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # 이메일 제거
    text = re.sub(r'\S+@\S+', '', text)
    
    # 사용자 아이디 제거 (예: @username)
    text = re.sub(r'@\w+', '', text)
    
    # 연속된 공백 제거
    text = re.sub(r'\s+', ' ', text)
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text


def normalize_emoticons(text: str) -> str:
    """
    이모티콘 정규화
    - 'ㅋㅋ', 'ㅎㅎ' 개수에 따라 감정도 표현
    - 'ㅠㅠ', 'ㅜㅜ' 슬픔 표현
    """
    # 긍정 이모티콘 (3개 이상은 강한 긍정)
    text = re.sub(r'ㅋ{3,}', '[강한긍정]', text)
    text = re.sub(r'ㅎ{3,}', '[강한긍정]', text)
    text = re.sub(r'ㅋ{1,2}', '[긍정]', text)
    text = re.sub(r'ㅎ{1,2}', '[긍정]', text)
    
    # 슬픔 이모티콘
    text = re.sub(r'ㅠ{2,}', '[슬픔]', text)
    text = re.sub(r'ㅜ{2,}', '[슬픔]', text)
    
    return text


def load_and_preprocess_data(file_path: str) -> pd.DataFrame:
    """
    데이터 로드 및 전처리
    """
    print("📂 데이터 로딩 중...")
    df = pd.read_csv(file_path, low_memory=False)
    print(f"   ✅ 원본 데이터: {len(df):,}개 샘플")
    
    # 필수 컬럼 확인
    required_cols = ['text', 'emotion', 'label_id']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"필수 컬럼 누락: {missing_cols}")
    
    # 텍스트 정제
    print("\n🧹 텍스트 정제 중...")
    df['text'] = df['text'].apply(clean_text)
    df['text'] = df['text'].apply(normalize_emoticons)
    
    # 빈 텍스트 제거
    before_len = len(df)
    df = df[df['text'].str.len() > 0].copy()
    after_len = len(df)
    print(f"   ✅ 빈 텍스트 제거: {before_len - after_len}개 샘플 제거")
    
    # label_id가 숫자가 아닌 경우 처리
    if df['label_id'].dtype == 'object':
        # emotion 컬럼을 사용하여 label_id 생성
        df['label_id'] = df['emotion'].map(LABEL2ID)
    
    # NaN 제거
    df = df.dropna(subset=['text', 'emotion', 'label_id']).copy()
    
    # label_id를 정수형으로 변환
    df['label_id'] = df['label_id'].astype(int)
    
    print(f"\n📊 최종 데이터: {len(df):,}개 샘플")
    
    return df


In [ ]:
# 데이터 로드 및 전처리
df = load_and_preprocess_data(DATA_PATH)

# 감정 분포 확인
print("\n" + "="*60)
print("📊 감정 분포")
print("="*60)
emotion_counts = df['emotion'].value_counts()
for emotion, count in emotion_counts.items():
    percentage = count / len(df) * 100
    print(f"{emotion:10s}: {count:6,}개 ({percentage:5.2f}%)")

# 시각화
plt.figure(figsize=(10, 6))
emotion_counts.plot(kind='bar', color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A', '#98D8C8'])
plt.title('감정 분포', fontsize=16, fontweight='bold')
plt.xlabel('감정', fontsize=12)
plt.ylabel('샘플 수', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## ⚖️ 6. 데이터 불균형 해소


In [ ]:
def balance_dataset(df: pd.DataFrame, strategy: str = "balanced") -> pd.DataFrame:
    """
    데이터 불균형 해소
    
    Args:
        df: 데이터프레임
        strategy: 'balanced' (균형), 'minority' (소수 클래스 기준), 'majority' (다수 클래스 기준), 또는 비율 딕셔너리
    """
    print("\n⚖️ 데이터 불균형 해소 중...")
    
    # 현재 분포
    print("\n📊 샘플링 전 분포:")
    before_counts = df['label_id'].value_counts().sort_index()
    for label_id, count in before_counts.items():
        emotion = ID2LABEL[label_id]
        print(f"   {emotion:10s} (ID {label_id}): {count:6,}개")
    
    # 가장 적은 클래스의 샘플 수 확인
    min_samples = before_counts.min()
    max_samples = before_counts.max()
    mean_samples = int(before_counts.mean())
    
    print(f"\n   최소: {min_samples:,}개, 최대: {max_samples:,}개, 평균: {mean_samples:,}개")
    print(f"   불균형 비율: {max_samples/min_samples:.2f}:1")
    
    # 샘플링 전략 설정
    if strategy == "balanced":
        # 평균보다 적은 클래스만 평균으로 맞춤 (오버샘플링만 가능하므로)
        # 평균보다 많은 클래스는 그대로 유지
        sampling_strategy = {}
        for label_id, count in before_counts.items():
            if count < mean_samples:
                # 평균보다 적으면 평균으로 맞춤
                sampling_strategy[label_id] = mean_samples
            # 평균보다 많거나 같으면 sampling_strategy에 포함하지 않음 (그대로 유지)
    elif strategy == "minority":
        # 소수 클래스 기준으로 맞춤 (모든 클래스를 최소 샘플 수로)
        # 하지만 오버샘플링만 가능하므로, 최소보다 적은 클래스만 최소로 맞춤
        sampling_strategy = {}
        for label_id, count in before_counts.items():
            if count < min_samples:
                sampling_strategy[label_id] = min_samples
    elif strategy == "majority":
        # 다수 클래스 기준으로 맞춤 (모든 클래스를 최대 샘플 수로)
        sampling_strategy = {label_id: max_samples for label_id in before_counts.index}
    else:
        # 사용자 정의 딕셔너리인 경우, 평균보다 적은 클래스만 필터링
        sampling_strategy = {}
        for label_id, target_count in strategy.items():
            current_count = before_counts[label_id]
            if target_count >= current_count:
                # 오버샘플링만 가능하므로, 현재보다 크거나 같을 때만 적용
                sampling_strategy[label_id] = target_count
            else:
                print(f"   ⚠️ {ID2LABEL[label_id]} (ID {label_id}): {current_count}개 → {target_count}개로 줄일 수 없습니다. 현재 샘플 수 유지.")
    
    if not sampling_strategy:
        print("\n   ⚠️ 샘플링할 클래스가 없습니다. 모든 클래스가 이미 충분한 샘플을 가지고 있습니다.")
        return df
    
    print(f"\n   📝 샘플링 대상: {len(sampling_strategy)}개 클래스")
    for label_id, target_count in sampling_strategy.items():
        current_count = before_counts[label_id]
        emotion = ID2LABEL[label_id]
        print(f"      {emotion:10s} (ID {label_id}): {current_count:6,}개 → {target_count:6,}개")
    
    # Oversampling (부족한 클래스 증강)
    oversampler = RandomOverSampler(
        sampling_strategy=sampling_strategy,
        random_state=RANDOM_SEED
    )
    
    # 텍스트는 그대로 두고 label_id만 사용하여 샘플링
    X = df[['text']].values
    y = df['label_id'].values
    
    X_resampled, y_resampled = oversampler.fit_resample(X, y)
    
    # 데이터프레임 재구성
    df_balanced = pd.DataFrame({
        'text': X_resampled.flatten(),
        'label_id': y_resampled
    })
    
    # emotion 컬럼 추가
    df_balanced['emotion'] = df_balanced['label_id'].map(ID2LABEL)
    
    # 샘플링 후 분포
    print("\n📊 샘플링 후 분포:")
    after_counts = df_balanced['label_id'].value_counts().sort_index()
    for label_id, count in after_counts.items():
        emotion = ID2LABEL[label_id]
        print(f"   {emotion:10s} (ID {label_id}): {count:6,}개")
    
    print(f"\n   총 샘플 수: {len(df_balanced):,}개 (이전: {len(df):,}개)")
    
    return df_balanced


# 불균형 해소 적용
if USE_SAMPLING:
    df = balance_dataset(df, strategy=SAMPLING_STRATEGY)
else:
    print("\n⚠️ 샘플링을 사용하지 않습니다.")


## 🤖 7. 모델 및 토크나이저 로드


In [ ]:
print(f"📥 모델 로딩 중: {MODEL_NAME}")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print("✅ 토크나이저 로드 완료")

# 모델 로드
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label=ID2LABEL,
    label2id=LABEL2ID
)
print("✅ 모델 로드 완료")

# GPU 사용 가능 시 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(f"✅ 모델을 {device}로 이동 완료")

print(f"\n📊 모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")


## 📈 8. 평가 메트릭 함수 정의


In [ ]:
def compute_metrics(eval_pred):
    """평가 메트릭 계산"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    
    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


def print_classification_report(y_true, y_pred, labels=None):
    """분류 리포트 출력"""
    if labels is None:
        labels = [ID2LABEL[i] for i in range(NUM_LABELS)]
    
    print("\n" + "="*60)
    print("📊 분류 리포트")
    print("="*60)
    print(classification_report(y_true, y_pred, target_names=labels, digits=4))
    
    # 혼동 행렬
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=labels, yticklabels=labels)
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.ylabel('실제', fontsize=12)
    plt.xlabel('예측', fontsize=12)
    plt.tight_layout()
    plt.show()


## 🔀 9. K-Fold 교차 검증 학습


In [ ]:
# 데이터 준비
texts = df['text'].tolist()
labels = df['label_id'].tolist()

# K-Fold 분할
skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_SEED)

# 결과 저장
fold_results = []
all_predictions = []
all_labels = []

print(f"🚀 K-Fold 교차 검증 시작 ({N_FOLDS} folds)\n")
print("="*60)

for fold, (train_idx, val_idx) in enumerate(skf.split(texts, labels), 1):
    print(f"\n{'='*60}")
    print(f"📁 Fold {fold}/{N_FOLDS}")
    print(f"{'='*60}")
    
    # 학습/검증 데이터 분할
    train_texts = [texts[i] for i in train_idx]
    train_labels = [labels[i] for i in train_idx]
    val_texts = [texts[i] for i in val_idx]
    val_labels = [labels[i] for i in val_idx]
    
    print(f"\n📊 데이터 분할:")
    print(f"   학습: {len(train_texts):,}개")
    print(f"   검증: {len(val_texts):,}개")
    
    # HuggingFace Dataset으로 변환
    train_dataset = HFDataset.from_dict({
        'text': train_texts,
        'label': train_labels
    })
    
    val_dataset = HFDataset.from_dict({
        'text': val_texts,
        'label': val_labels
    })
    
    # 토큰화
    def tokenize_function(examples):
        return tokenizer(
            examples['text'],
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH
        )
    
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    
    # 라벨 설정
    train_dataset = train_dataset.rename_column('label', 'labels')
    val_dataset = val_dataset.rename_column('label', 'labels')
    
    train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
    
    # 모델 재초기화 (각 fold마다 새로 학습)
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME,
        num_labels=NUM_LABELS,
        id2label=ID2LABEL,
        label2id=LABEL2ID
    )
    
    # 학습 인자 설정
    training_args = TrainingArguments(
        output_dir=f'./checkpoints/fold_{fold}',
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE * 2,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        warmup_steps=WARMUP_STEPS,
        logging_dir=f'./logs/fold_{fold}',
        logging_steps=100,
        eval_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='f1',
        greater_is_better=True,
        save_total_limit=2,
        seed=RANDOM_SEED,
        fp16=torch.cuda.is_available(),  # GPU 사용 시 혼합 정밀도
        report_to='none',  # wandb 등 사용 시 'wandb'로 변경
    )
    
    # Trainer 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
    )
    
    # 학습
    print(f"\n🎓 학습 시작...")
    train_result = trainer.train()
    
    # 평가
    print(f"\n📊 평가 중...")
    eval_result = trainer.evaluate()
    
    # 예측
    predictions = trainer.predict(val_dataset)
    y_pred = np.argmax(predictions.predictions, axis=1)
    
    # 결과 저장
    fold_results.append({
        'fold': fold,
        'accuracy': eval_result['eval_accuracy'],
        'f1': eval_result['eval_f1'],
        'precision': eval_result['eval_precision'],
        'recall': eval_result['eval_recall']
    })
    
    all_predictions.extend(y_pred.tolist())
    all_labels.extend(val_labels)
    
    print(f"\n✅ Fold {fold} 완료!")
    print(f"   Accuracy: {eval_result['eval_accuracy']:.4f}")
    print(f"   F1 Score: {eval_result['eval_f1']:.4f}")
    print(f"   Precision: {eval_result['eval_precision']:.4f}")
    print(f"   Recall: {eval_result['eval_recall']:.4f}")
    
    # 메모리 정리
    del model, trainer, train_dataset, val_dataset
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

print(f"\n{'='*60}")
print(f"🎉 K-Fold 교차 검증 완료!")
print(f"{'='*60}")


## 📊 10. 결과 분석 및 시각화


In [ ]:
# Fold별 결과 요약
results_df = pd.DataFrame(fold_results)

print("\n" + "="*60)
print("📊 K-Fold 결과 요약")
print("="*60)
print(results_df.to_string(index=False))

print("\n" + "="*60)
print("📈 평균 성능")
print("="*60)
print(f"Accuracy: {results_df['accuracy'].mean():.4f} (±{results_df['accuracy'].std():.4f})")
print(f"F1 Score:  {results_df['f1'].mean():.4f} (±{results_df['f1'].std():.4f})")
print(f"Precision: {results_df['precision'].mean():.4f} (±{results_df['precision'].std():.4f})")
print(f"Recall:    {results_df['recall'].mean():.4f} (±{results_df['recall'].std():.4f})")

# 시각화
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = ['accuracy', 'f1', 'precision', 'recall']
for idx, metric in enumerate(metrics):
    ax = axes[idx // 2, idx % 2]
    ax.bar(range(1, N_FOLDS + 1), results_df[metric], color='#4ECDC4', alpha=0.7)
    ax.axhline(y=results_df[metric].mean(), color='r', linestyle='--', label='평균')
    ax.set_title(f'{metric.upper()}', fontsize=14, fontweight='bold')
    ax.set_xlabel('Fold', fontsize=12)
    ax.set_ylabel('Score', fontsize=12)
    ax.set_xticks(range(1, N_FOLDS + 1))
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# 전체 예측 결과 분석
print("\n" + "="*60)
print("📊 전체 예측 결과 (모든 Fold 통합)")
print("="*60)

overall_accuracy = accuracy_score(all_labels, all_predictions)
overall_f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"\n전체 Accuracy: {overall_accuracy:.4f}")
print(f"전체 F1 Score: {overall_f1:.4f}")

# 분류 리포트
print_classification_report(all_labels, all_predictions)


## 💾 11. 최종 모델 저장 (선택사항)

가장 성능이 좋은 Fold의 모델을 저장하거나, 모든 Fold 모델의 앙상블을 사용할 수 있습니다.


In [ ]:
# 가장 성능이 좋은 Fold 찾기
best_fold = results_df.loc[results_df['f1'].idxmax(), 'fold']
print(f"🏆 최고 성능 Fold: {int(best_fold)} (F1: {results_df.loc[results_df['f1'].idxmax(), 'f1']:.4f})")

# 최고 성능 모델 로드 및 저장
# 각 fold의 마지막 체크포인트를 찾아서 저장
import glob

best_fold_dir = f'./checkpoints/fold_{int(best_fold)}'
checkpoint_dirs = sorted(glob.glob(f'{best_fold_dir}/checkpoint-*'), 
                         key=lambda x: int(x.split('-')[-1]))

if checkpoint_dirs:
    best_model_path = checkpoint_dirs[-1]  # 마지막 체크포인트
    print(f"\n💾 최고 성능 모델 저장 중...")
    print(f"   모델 경로: {best_model_path}")
    
    # 모델 로드
    final_model = AutoModelForSequenceClassification.from_pretrained(best_model_path)
    final_tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    
    # 저장
    save_path = './best_emotion_model'
    final_model.save_pretrained(save_path)
    final_tokenizer.save_pretrained(save_path)
    
    print(f"✅ 모델 저장 완료: {save_path}")
    print(f"\n📝 사용 방법:")
    print(f"   from transformers import AutoTokenizer, AutoModelForSequenceClassification")
    print(f"   tokenizer = AutoTokenizer.from_pretrained('{save_path}')")
    print(f"   model = AutoModelForSequenceClassification.from_pretrained('{save_path}')")
else:
    print(f"⚠️ 모델 체크포인트를 찾을 수 없습니다: {best_fold_dir}")


## 🧪 12. 테스트 예제


In [ ]:
# 저장된 모델이 있는 경우 테스트
if os.path.exists('./best_emotion_model'):
    # 모델 로드
    test_model = AutoModelForSequenceClassification.from_pretrained('./best_emotion_model')
    test_tokenizer = AutoTokenizer.from_pretrained('./best_emotion_model')
    test_model.to(device)
    test_model.eval()
    
    # 테스트 문장들
    test_sentences = [
        "오늘 정말 기분이 좋아! 회사에서 프로젝트가 성공했어.",
        "면접에서 떨어져서 너무 슬프고 힘들어.",
        "내일 면접인데 너무 불안하고 걱정돼.",
        "상사가 자꾸 야근을 시켜서 정말 화가 나!",
        "오늘 날씨가 좋네요. 평범한 하루입니다."
    ]
    
    print("\n" + "="*60)
    print("🧪 감정 분석 테스트")
    print("="*60)
    
    for sentence in test_sentences:
        # 토큰화
        inputs = test_tokenizer(
            sentence,
            truncation=True,
            padding='max_length',
            max_length=MAX_LENGTH,
            return_tensors='pt'
        ).to(device)
        
        # 예측
        with torch.no_grad():
            outputs = test_model(**inputs)
            predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            predicted_id = predictions.argmax().item()
            confidence = predictions[0][predicted_id].item()
        
        predicted_emotion = ID2LABEL[predicted_id]
        
        print(f"\n문장: {sentence}")
        print(f"예측: {predicted_emotion} (신뢰도: {confidence:.4f})")
        print(f"   {dict(zip([ID2LABEL[i] for i in range(NUM_LABELS)], [f'{p:.4f}' for p in predictions[0].cpu().numpy()]))}")
else:
    print("⚠️ 저장된 모델이 없습니다. 먼저 모델을 학습하고 저장하세요.")


## 📝 13. 요약 및 개선 사항

### 성능 개선 팁:

1. **하이퍼파라미터 튜닝**:
   - Learning Rate: 1e-5 ~ 5e-5 범위에서 실험
   - Batch Size: GPU 메모리에 맞게 조정 (16, 32, 64)
   - Max Length: 128 또는 256으로 실험

2. **데이터 증강**:
   - Back-translation (한국어 → 영어 → 한국어)
   - 동의어 교체
   - 문장 순서 변경

3. **앙상블**:
   - 여러 모델 (KoBERT, KoELECTRA, KC-BERT) 조합
   - 다수결 투표 또는 평균 확률 사용

4. **고급 기법**:
   - Focal Loss (불균형 데이터용)
   - Class Weight 조정
   - 더 큰 모델 사용 (KoBERT-large 등)

---

**작성일**: 2025년  
**모델**: KoBERT (monologg/kobert)  
**목적**: 한국어 감정 분석 최적화
